In [1]:
%matplotlib inline

DEEP Q LEARNING CAR RACING IMPLEMENTATION
----------------

In [2]:
import gym
import math
import random
import numpy as np
import matplotlib
matplotlib.use('Qt5Agg') 
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


env = gym.make('CarRacing-v0')


# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# set manual seed for reproducability
torch.manual_seed(360)

/home/peter/Documents/gym/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


**Replay Memory:**

Define a transition in our environment, mapping (state, action) to (next_state, reward).

Define Replay Memory which is a bounded size buffer holding recently observed transitions.


In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

**Model:**

Define our model, this has been completely overhauled from the original tutorial's model.


In [4]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(288, 124)
        self.fc2  = nn.Linear(124, 3)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = F.relu(self.fc1(x.view(x.size(0), -1)))
        x = self.fc2(x)
        return x

**Data Processing:**

Extract the next screen from the environment and crop it down, centered around the cart.
This removes a lot of unessesary information, allowing our model to train more efficiently


In [ ]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize((40, 40), interpolation=Image.CUBIC),
                    T.ToTensor()])


def get_cart_location(screen_width):
    world_width = env.x_threshold * 2
    scale = screen_width / world_width
    return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART

def get_screen(screen):
    # Returned screen requested by gym is 400x600x3, but is sometimes larger
    # such as 800x1200x3. Transpose it into torch order (CHW).
    # screen = env.render(mode='rgb_array')
    # env.render()
    
    # Convert to grayscale (weights based off CCIR 601)
    screen = screen[:,:,0] * 0.2989 + screen[:,:,1] * 0.5870 + screen[:,:,2] * 0.1140
    # screen = screen[:,:,0] * 0.33 + screen[:,:,1] * 0.33 + screen[:,:,2] * 0.33
    
    # Crop out the bottom score 
    screen = screen[0:-50,:]
    
    # Crop the screen even more
    [height, width] = screen.shape
    screen = screen[0:int(height*1.0), int(width*0.25):int(width*0.75)]
    
    # Set the screen values to either black or white
    screen[screen < 125] = 0
    screen[screen > 125] = 255
    
    # Convert to float, and then to torch tensor
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    
    # Resize (reducing dimensionality), and add a batch dimension (BHW)
    return resize(screen).unsqueeze(0).to(device)

env.reset()
plt.figure()
for x in range(50):
    plt.clf()
    plt.imshow(get_screen(env.render(mode='rgb_array')).cpu().squeeze(0).numpy()[0],
               cmap='gray', 
               vmin=0, vmax=1)
    plt.title('Example extracted screen')
    plt.show()

    plt.pause(0.001) 
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

    env.step([0, 0.2, 0])

Track generation: 1125..1421 -> 296-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1027..1286 -> 259-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1346..1687 -> 341-tiles track


**Training:**

This initiallizes our policy and target_nets as well as defines some helper functions.

select_action will be used to select the next action we will take. This uses an epsilon greedy
policy, creating a threshold to determine whether we 'exploit' our model or 'explore' the state 
space.

plot_durations allows us to plot our results as the model is training.


In [ ]:
BATCH_SIZE = 64
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.001
EPS_DECAY =1000
TARGET_UPDATE = 10

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
init_screen = get_screen(env.render(mode='rgb_array'))
_, _, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
n_actions = env.action_space.shape[0]

policy_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state) #.max(1)[1].view(1, 1)
    else:
        return torch.tensor(env.action_space.sample(), device=device, dtype=torch.float)


episode_durations = []


def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())
        return means.numpy()

optimize_model performs a single step of optimization in our training. This will essentially compute 
the loss of the model and optimize accordingly.


In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    # print("state", batch.state)
    # print("action", batch.action)
    # batch.action.double()
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch) #.gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # for param in policy_net.parameters():
    #     param.grad.data.clamp_(-1, 1)
    optimizer.step()

**Training loop:**

This is the rest of our training loop, we use the policy_net for selecting actions while using the
target_net for loss calculations as it is more stable. We update the target_net's parameters with that
of the policy_net every 10 episodes. by having the loop set up like this, we are able to continually run
this block in our notebook, extending the training so we don't need to guess how many episodes is needed
to see success.


In [ ]:
num_episodes = 20
for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    last_screen = get_screen(env.render(mode='rgb_array'))
    current_screen = get_screen(env.render(mode='rgb_array'))
    state = current_screen - last_screen
    
    total_reward = 0.0
    for t in count():        
        # Select and perform an action
        action = select_action(state).cpu().numpy()
        action = np.clip(action, env.action_space.low, env.action_space.high)
        if action.shape[0] == 1:
            action = action[0]
        observation, reward, done, _ = env.step(action)
        total_reward += reward
        
        action = torch.tensor(action, device=device)
        reward = torch.tensor([reward], device=device)

        # Observe new state
        last_screen = current_screen
        current_screen = get_screen(observation)
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action.float(), next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_durations.append(total_reward + 1)
            total_reward = 0
            mean = plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()